In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import sys, traceback
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier

In [2]:

data = pd.read_csv("./data/vector_allocation3_1_Com_FINAL_WITHOUT_DUPLICATE_NEW_SORTED_COMBINED.csv")

col_names = data.columns.to_list()

print("Columns Names: ", col_names)

print("\nSample data:")

data.head()


Columns Names:  ['URLID', 'stypeID', 'r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35', 'r36', 'r37', 'r38', 'r39', 'r40', 'r41', 'r42', 'r43', 'r44', 'r45', 'r46', 'r47', 'r48', 'r49', 'r50', 'r51', 'r52', 'r53', 'r54', 'r55', 'r56', 'r57', 'r58', 'r59', 'r60', 'r61', 'r62', 'r63', 'r64', 'r65', 'r66', 'r67', 'r68', 'r69', 'r70', 'r71', 'r72', 'r73', 'r74', 'r75', 'r76', 'r77', 'r78', 'r79', 'r80', 'r81', 'r82', 'r83', 'r84', 'r85', 'r86', 'r87', 'r88', 'r89', 'r90', 'r91', 'r92', 'r93', 'r94', 'r95', 'r96', 'r97', 'r98', 'r99', 'r100', 'r101', 'r102', 'r103', 'r104', 'r105', 'r106', 'r107', 'r108', 'r109', 'r110', 'r111', 'attack_status_value', 'vectors_class_output', 'attack_vector_class', 'attack_vector_type']

Sample data:


,URLID,stypeID,r0,r1,r2,r3,r4,r5,r6,r7,...,r106,r107,r108,r109,r110,r111,attack_status_value,vectors_class_output,attack_vector_class,attack_vector_type
0,1,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,No,0,BENIGN
1,2,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,No,0,BENIGN
2,3,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,No,0,BENIGN
3,4,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,No,0,BENIGN
4,5,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,No,0,BENIGN


In [12]:
# DISPLAY DATATYPWE FOR EACH COLUMJS

data.dtypes

URLID                    int64
stypeID                  int64
r0                       int64
r1                       int64
r2                       int64
                         ...  
r111                     int64
attack_status_value      int64
vectors_class_output    object
attack_vector_class      int64
attack_vector_type      object
Length: 118, dtype: object

### CHECK FOR MISSING VALUES

In [18]:
data.isnull().any().sum()

# No missing values on dataset

0

In [10]:
data.attack_vector_class.describe()

count    102186.000000
mean          8.829830
std           4.719239
min           0.000000
25%           6.000000
50%           8.000000
75%          12.000000
max          18.000000
Name: attack_vector_class, dtype: float64

## FEATURE SELECTION

In [19]:
data_independent = data.columns.values.tolist()


_determiner = [i for i in data_independent if i not in ["URLID","stypeID", "attack_vector_type", "vectors_class_output", "attack_vector_class"]]



In [24]:
# Lets perform Feature Selection (Dimensionality Reduction) on our dataset


# Create our X (all the fearures except Y {feature matrix})
X = data.drop(["URLID","stypeID", "attack_vector_type", "vectors_class_output", "attack_vector_class"], axis=1)

Y = data["attack_vector_class"]


from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier 


model = RandomForestClassifier() #Using the default hyper params

rfe = RFE(model)

rfe = rfe.fit(X, Y)

(rfe.support_)

print(rfe.ranking_)



[ True  True  True  True  True False  True  True False False  True  True
 False  True  True  True False  True False False  True  True  True  True
  True False  True False False  True False False False  True  True  True
 False False False  True  True  True  True False False  True  True  True
  True False  True False False  True  True False False False False False
  True False False False False  True False False False  True False  True
  True False  True  True  True False  True  True False False False False
 False False  True  True  True  True False  True False False  True False
 False False  True  True False False  True False False False False  True
 False False False  True  True]
[ 1  1  1  1  1 22  1  1  7 11  1  1  6  1  1  1 31  1 47 40  1  1  1  1
  1 39  1 42 38  1 48 12  5  1  1  1 37 17 33  1  1  1  1 44 41  1  1  1
  1 53  1 57 56  1  1 15 19 36 34 32  1 10  9 46 18  1 51 50 43  1  3  1
  1 21  1  1  1  2  1  1 20  4 30 24 25 55  1  1  1  1 26  1 27 28  1 29
 14 35  1  1 45 52 

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r103,r104,r105,r106,r107,r108,r109,r110,r111,attack_status_value
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102181,1,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
102182,1,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
102183,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
102184,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
